<a href="https://colab.research.google.com/github/acreatorrr/GAN_Deepfake_detection/blob/main/Deepfake_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')


/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os

In [17]:



image_dir_real = '/content/gdrive/MyDrive/Deepfake-detection/data/Real'
image_dir_fake = '/content/gdrive/MyDrive/Deepfake-detection/data/DeepFake'


def preprocess_image(image):
    image = tf.image.resize(image, (128, 128))
    image = image / 255.0
    return image

def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = preprocess_image(image)
    return image


real_image_paths = [os.path.join(image_dir_real, filename) for filename in os.listdir(image_dir_real) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]
fake_image_paths = [os.path.join(image_dir_fake, filename) for filename in os.listdir(image_dir_fake) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]


real_images = [load_image(path) for path in real_image_paths]


fake_images = [load_image(path) for path in fake_image_paths]


In [18]:
def create_discriminator_model(input_shape=(128, 128, 3)):
    model = keras.Sequential([
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same", input_shape=input_shape),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        layers.Flatten(),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

discriminator = create_discriminator_model()
discriminator.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(0.0002, 0.5), metrics=["accuracy"])


In [19]:
X_real = np.array(real_images)
X_fake = np.array(fake_images)

y_real = np.ones((X_real.shape[0], 1))
y_fake = np.zeros((X_fake.shape[0], 1))

X_train = np.concatenate((X_real, X_fake), axis=0)
y_train = np.concatenate((y_real, y_fake), axis=0)

discriminator.fit(X_train, y_train, epochs=10, batch_size=16)


Epoch 1/10
444/444 [==============================] - 15s 8ms/step - loss: 0.1718 - accuracy: 0.9389
Epoch 2/10
444/444 [==============================] - 3s 7ms/step - loss: 0.0545 - accuracy: 0.9796
Epoch 3/10
444/444 [==============================] - 3s 7ms/step - loss: 0.0464 - accuracy: 0.9842
Epoch 4/10
444/444 [==============================] - 3s 8ms/step - loss: 0.0276 - accuracy: 0.9918
Epoch 5/10
444/444 [==============================] - 4s 8ms/step - loss: 0.0105 - accuracy: 0.9969
Epoch 6/10
444/444 [==============================] - 3s 7ms/step - loss: 0.0109 - accuracy: 0.9968
Epoch 7/10
444/444 [==============================] - 3s 7ms/step - loss: 0.0225 - accuracy: 0.9952
Epoch 8/10
444/444 [==============================] - 3s 8ms/step - loss: 0.0167 - accuracy: 0.9939
Epoch 9/10
444/444 [==============================] - 4s 8ms/step - loss: 0.0305 - accuracy: 0.9934
Epoch 10/10
444/444 [==============================] - 3s 7ms/step - loss: 0.0059 - accuracy: 0.997

In [27]:
def detect_deepfake_image(image, threshold=0.5):
    image = np.expand_dims(image, axis=0)
    prediction = discriminator.predict(image)

    if prediction > threshold:
        return "This image is likely a real."
    else:
        return "This image appears to be deepfake."

realImage='/content/gdrive/MyDrive/df_det/real/99_372.jpg'
fakeImage='/content/gdrive/MyDrive/df_det/fake/179_96.jpg'

image_to_check = load_image('/content/deep_fake_test.jpg')
result = detect_deepfake_image(image_to_check)
print(result)


1/1 [==============================] - 0s 184ms/step
This image appears to be deepfake.
